In [1]:
!pip install transformers
!pip install datasets
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.1 MB/s eta 0:00:0

In [2]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import InputExample, InputFeatures
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [4]:
#code to work with data in google collab
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive"

Mounted at /content/drive
/content/drive/MyDrive


In [5]:
df = pd.read_csv("preprocessed_data.csv")

In [6]:
df = df[["reviewText", "LABEL_COLUMN"]]
df = df.rename(columns={"reviewText":"DATA_COLUMN"})
df.head()

,DATA_COLUMN,LABEL_COLUMN
0,Heerlijk eten en leuke sfeer. Veel keuze bij h...,positive
1,Met z'n vieren dit restaurant bezocht. We hadd...,positive
2,Wait to be seated! We moesten lang wachten.Ont...,negative
3,Wij zijn volgens ons afspraak om 18.00 uur daa...,negative
4,Heerlijk gegeten!,positive


In [7]:
from transformers import AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

In [8]:
def process_data(row):

    text = row['DATA_COLUMN']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding=True, truncation=True, max_length=128)

    label = 0
    if row['LABEL_COLUMN'] == 'positive':
        label += 1

    encodings['label'] = int(label)
    encodings['text'] = text

    return encodings

In [9]:
print(process_data({
    'DATA_COLUMN': 'this is a sample review of a movie.',
    'LABEL_COLUMN': 'positive'
}))

{'input_ids': [0, 23712, 12, 410, 23943, 437, 5025, 25, 410, 16578, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 1, 'text': 'this is a sample review of a movie.'}


In [10]:
processed_data = []
df = df.sample(100000,random_state=123)

for i in range(len(df)):
    processed_data.append(process_data(df.iloc[i]))

In [11]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

new_df = pd.DataFrame(processed_data)
#columns_titles = ["attention_mask","input_ids","token_type_ids", "label", "text"]
#new_df=new_df.reindex(columns=columns_titles)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [12]:
import pyarrow as pa
import tensorflow as tf
from datasets import Dataset


train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))



In [13]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "pdelobelle/robbert-v2-dutch-base",
    num_labels=2
)

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=128,  # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer_robbert = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

In [15]:
trainer_robbert.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.057200,0.043983
2,0.026800,0.047407
3,0.001900,0.053593


TrainOutput(global_step=3750, training_loss=0.03829862883749108, metrics={'train_runtime': 5307.177, 'train_samples_per_second': 45.222, 'train_steps_per_second': 0.707, 'total_flos': 1.57866633216e+16, 'train_loss': 0.03829862883749108, 'epoch': 3.0})

In [16]:
trainer_robbert.evaluate()

{'eval_loss': 0.0535925030708313,
 'eval_runtime': 131.5291,
 'eval_samples_per_second': 152.058,
 'eval_steps_per_second': 1.194,
 'epoch': 3.0}

In [18]:

model.save_pretrained('./robbert_100k/')

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./robbert_100k/').to(device)

In [ ]:
predictions = trainer_robbert.predict(valid_hg)

In [ ]:
import math

probabilities = pd.DataFrame(predictions[0])
#probabilities[0]
probabilities["exp_1"] = probabilities[0].apply(lambda x: math.exp(x))
probabilities["exp_2"] = probabilities[1].apply(lambda x: math.exp(x))
probabilities["sum"] = probabilities["exp_1"] + probabilities["exp_2"]
probabilities["final_1"] = probabilities["exp_1"] / probabilities["sum"]
probabilities["final_2"] = probabilities["exp_2"] / probabilities["sum"]
probabilities = probabilities[["final_1", "final_2"]]
probabilities = probabilities.rename(columns = {"final_1":0,"final_2":1})


In [ ]:
probabilities

In [ ]:
predicted = probabilities.idxmax(axis=1)
predicted

In [ ]:
actual = predictions[1]

In [ ]:
# confusion matrix: actual vs predicted counts
from sklearn.metrics import confusion_matrix
confmat  =  confusion_matrix(actual,predicted)
confmat

In [ ]:
# derive True Positive, False Positive, False Negative and True Negative from confusion matrix
TP = confmat[1,1]
FP = confmat[0,1]
FN = confmat[1,0]
TN = confmat[0,0]

In [ ]:
accuracy = (TP+TN)/(TP+FP+FN+TN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
F1 = 2*TP / (2*TP+FP+FN)
print("accuracy = ",accuracy)
print("precision = ", precision)
print("recall = ", recall)
print("F1-score = ", F1)